In [1]:
#Импорт библиотек
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import pickle
import time
from tqdm import tqdm
from scipy import stats
import pprint

from matplotlib.ticker import FormatStrFormatter

# 1. Загрузка данных

In [2]:
df_sessions = pd.read_csv('data/ga_sessions.csv')
print(df_sessions.shape)
df_sessions.head()

C:\Users\Denis\AppData\Local\Temp\ipykernel_10248\2466688719.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sessions = pd.read_csv('data/ga_sessions.csv')


(1860042, 18)


,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,9055434745589932991.1637753792.1637753792,2108382700.163776,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust
1,905544597018549464.1636867290.1636867290,210838531.163687,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow
2,9055446045651783499.1640648526.1640648526,2108385331.164065,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk
3,9055447046360770272.1622255328.1622255328,2108385564.162225,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow
4,9055447046360770272.1622255345.1622255345,2108385564.162225,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow


In [3]:
df_hits = pd.read_csv('data/ga_hits.csv')
print(df_hits.shape)
df_hits.head()

(15726470, 11)


,session_id,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value
0,5639623078712724064.1640254056.1640254056,2021-12-23,597864.0,30,event,NaN,sberauto.com/cars?utm_source_initial=google&ut...,quiz,quiz_show,NaN,NaN
1,7750352294969115059.1640271109.1640271109,2021-12-23,597331.0,41,event,NaN,sberauto.com/cars/fiat?city=1&city=18&rental_c...,quiz,quiz_show,NaN,NaN
2,885342191847998240.1640235807.1640235807,2021-12-23,796252.0,49,event,NaN,sberauto.com/cars/all/volkswagen/polo/e994838f...,quiz,quiz_show,NaN,NaN
3,142526202120934167.1640211014.1640211014,2021-12-23,934292.0,46,event,NaN,sberauto.com/cars?utm_source_initial=yandex&ut...,quiz,quiz_show,NaN,NaN
4,3450086108837475701.1640265078.1640265078,2021-12-23,768741.0,79,event,NaN,sberauto.com/cars/all/mercedes-benz/cla-klasse...,quiz,quiz_show,NaN,NaN


In [4]:
#Объеденим два датасета в один по колонке 'session_id'
df_full = df_sessions.merge(df_hits, on = 'session_id', how = 'inner')
print(df_full.shape)
df_full.head()

(15685219, 28)


,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,...,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value
0,9055434745589932991.1637753792.1637753792,2108382700.163776,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,...,2021-11-24,3665.0,3,event,NaN,podpiska.sberauto.com/,sub_page_view,sub_landing,NaN,NaN
1,9055434745589932991.1637753792.1637753792,2108382700.163776,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,...,2021-11-24,46592.0,4,event,NaN,podpiska.sberauto.com/,sub_button_click,sub_view_cars_click,vodKSlUobUWTVlgsJqdI,NaN
2,905544597018549464.1636867290.1636867290,210838531.163687,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,...,2021-11-14,921.0,3,event,NaN,podpiska.sberauto.com/,sub_page_view,sub_landing,NaN,NaN
3,9055446045651783499.1640648526.1640648526,2108385331.164065,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,...,2021-12-28,84155.0,10,event,NaN,sberauto.com/cars?utm_source_initial=sbol&utm_...,search_form,search_form_region,KWTCzSIXzoqUWjfUQMgP,NaN
4,9055446045651783499.1640648526.1640648526,2108385331.164065,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,...,2021-12-28,102979.0,13,event,NaN,sberauto.com/cars?utm_source_initial=sbol&utm_...,search_form,search_form_region,CBqnBQaKoQUyWJhLcxxN,NaN


# 2.Data Preparation

In [5]:
df_clean = df_full.copy()
print(df_clean.shape)
df_clean.head()

(15685219, 28)


,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,...,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value
0,9055434745589932991.1637753792.1637753792,2108382700.163776,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,...,2021-11-24,3665.0,3,event,NaN,podpiska.sberauto.com/,sub_page_view,sub_landing,NaN,NaN
1,9055434745589932991.1637753792.1637753792,2108382700.163776,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,...,2021-11-24,46592.0,4,event,NaN,podpiska.sberauto.com/,sub_button_click,sub_view_cars_click,vodKSlUobUWTVlgsJqdI,NaN
2,905544597018549464.1636867290.1636867290,210838531.163687,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,...,2021-11-14,921.0,3,event,NaN,podpiska.sberauto.com/,sub_page_view,sub_landing,NaN,NaN
3,9055446045651783499.1640648526.1640648526,2108385331.164065,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,...,2021-12-28,84155.0,10,event,NaN,sberauto.com/cars?utm_source_initial=sbol&utm_...,search_form,search_form_region,KWTCzSIXzoqUWjfUQMgP,NaN
4,9055446045651783499.1640648526.1640648526,2108385331.164065,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,...,2021-12-28,102979.0,13,event,NaN,sberauto.com/cars?utm_source_initial=sbol&utm_...,search_form,search_form_region,CBqnBQaKoQUyWJhLcxxN,NaN


# 3.Data Cleaning

### 3.1 Проверка данных на дубликаты

In [6]:
df_clean[df_clean.duplicated()]
#Дубликатов нет

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,...,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value


### 3.2 Проверка данных на пропуски

In [7]:
print("Информация о выборке:\n ")
df_clean.info(null_counts=True)

Информация о выборке:
 


C:\Users\Denis\AppData\Local\Temp\ipykernel_10248\928169365.py:2: FutureWarning: null_counts is deprecated. Use show_counts instead
  df_clean.info(null_counts=True)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 15685219 entries, 0 to 15685218
Data columns (total 28 columns):
 #   Column                    Non-Null Count     Dtype  
---  ------                    --------------     -----  
 0   session_id                15685219 non-null  object 
 1   client_id                 15685219 non-null  object 
 2   visit_date                15685219 non-null  object 
 3   visit_time                15685219 non-null  object 
 4   visit_number              15685219 non-null  int64  
 5   utm_source                15684519 non-null  object 
 6   utm_medium                15685219 non-null  object 
 7   utm_campaign              13486346 non-null  object 
 8   utm_adcontent             12852801 non-null  object 
 9   utm_keyword               6480392 non-null   object 
 10  device_category           15685219 non-null  object 
 11  device_os                 6526837 non-null   object 
 12  device_brand              11739344 non-null  object 
 13  device_mod

In [8]:
missing_values = ((df_clean.isna().sum() / len(df_clean)) * 100).sort_values()
print("Процент пропущенных значений:\n ")
missing_values

Процент пропущенных значений:
 


session_id                    0.000000
hit_page_path                 0.000000
hit_type                      0.000000
hit_number                    0.000000
hit_date                      0.000000
geo_city                      0.000000
geo_country                   0.000000
device_browser                0.000000
device_screen_resolution      0.000000
event_category                0.000000
event_action                  0.000000
utm_medium                    0.000000
visit_number                  0.000000
visit_time                    0.000000
visit_date                    0.000000
client_id                     0.000000
device_category               0.000000
utm_source                    0.004463
utm_campaign                 14.018759
utm_adcontent                18.057880
event_label                  23.765572
device_brand                 25.156646
hit_referer                  39.753975
device_os                    58.388614
hit_time                     58.400224
utm_keyword              

In [9]:
df_clean = df_clean.drop(["visit_date", "visit_time", "device_model", 
                         "hit_date", "hit_time", "event_category", 
                         "event_label", "event_value", "device_screen_resolution",
                         "device_os", "visit_number", "hit_type",
                         "utm_adcontent","utm_keyword", "client_id"], axis=1)

#Удаление неинформативных колонок

In [10]:
df_clean = df_clean[df_clean["utm_source"].notna()]
#Удаление пустых строчек в колонках "utm_source"

In [11]:
df_clean.hit_referer = df_clean.hit_referer.fillna(df_clean.hit_referer.mode().tolist()[0])
df_clean.utm_campaign = df_clean.utm_campaign.fillna(df_clean.utm_campaign.mode().tolist()[0])
#Замена пустых строк в колонках "hit_referer", "utm_campaign" наиболее популярными значениями.

In [12]:
df_clean.device_brand = df_clean.device_brand.fillna("other")
#Замена пустых строк в колонке "device_brand" значением "Other".

In [13]:
missing_values = ((df_clean.isna().sum() / len(df_clean)) * 100).sort_values()
print("Процент пропущенных значений:\n ")
missing_values

Процент пропущенных значений:
 


session_id         0.0
utm_source         0.0
utm_medium         0.0
utm_campaign       0.0
device_category    0.0
device_brand       0.0
device_browser     0.0
geo_country        0.0
geo_city           0.0
hit_number         0.0
hit_referer        0.0
hit_page_path      0.0
event_action       0.0
dtype: float64

### 3.3 Формирование новых прзнаков

In [14]:
#Переназначение индексов
df_clean.reset_index(drop= True , inplace= True )
#df_clean.head()

In [15]:
#Упрощение названий браузеров до 1 слова
df_clean.device_browser = df_clean.device_browser.apply(lambda x: x.lower().split(' ')[0])

In [16]:
#Пропишем все навания городов с маленькой буквы
df_clean.geo_city = df_clean.geo_city.apply(lambda x: x.lower())

In [17]:
#Делим приход пользователей на тех что из соц.сетей и остальные
df_clean["target_city"] = pd.Series()
for i in tqdm(range(len(df_clean.geo_city))):
    if df_clean.geo_city[i] == "moscow": 
        df_clean["target_city"][i] = "moscow"
    elif df_clean.geo_city[i] != "saint petersburg":
        df_clean["target_city"][i] = "spb"
    else:
        df_clean["target_city"][i] = "other"
                      
print(df_clean.target_city.value_counts()) 

C:\Users\Denis\AppData\Local\Temp\ipykernel_10248\2531878530.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_clean["target_city"] = pd.Series()
  0%|          | 0/15684519 [00:00<?, ?it/s]C:\Users\Denis\AppData\Local\Temp\ipykernel_10248\2531878530.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["target_city"][i] = "spb"
100%|██████████| 15684519/15684519 [03:17<00:00, 79588.48it/s]


moscow    6794589
spb       6524444
other     2365486
Name: target_city, dtype: int64


In [18]:
#Добавим в значение ссылки приписку "none/none/none/none" для выделения марок и моделей автомобиля.
df_clean["new_hit_page_path"] = pd.Series()
for i in tqdm(range(len(df_clean.hit_page_path))):
    df_clean["new_hit_page_path"][i] = df_clean.hit_page_path[i] + "none/none/none/none" 
    
#Добавление нового признака "car_brand"
df_clean["car_brand"] = df_clean.new_hit_page_path.apply(lambda x: x.lower().split('/')[3].split('?')[0])
#Добавление нового признака "car_model"
df_clean["car_model"] = df_clean.new_hit_page_path.apply(lambda x: x.lower().split('/')[4].split('?')[0])

#Удалим колонки с ссылками, они более не нужны.
df_clean = df_clean.drop(["hit_page_path", "new_hit_page_path"], axis=1)
print(df_clean.shape)
print(df_clean.car_brand.value_counts())

C:\Users\Denis\AppData\Local\Temp\ipykernel_10248\1822752888.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_clean["new_hit_page_path"] = pd.Series()
  0%|          | 0/15684519 [00:00<?, ?it/s]C:\Users\Denis\AppData\Local\Temp\ipykernel_10248\1822752888.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["new_hit_page_path"][i] = df_clean.hit_page_path[i] + "none/none/none/none"
100%|██████████| 15684519/15684519 [02:39<00:00, 98233.09it/s] 


(15684519, 15)
none             12185306
skoda              742249
mercedes-benz      471133
volkswagen         414374
lada-vaz           402182
                   ...   
insignia                1
bszfi+                  1
cs75                    1
teramont                1
2106                    1
Name: car_brand, Length: 623, dtype: int64


In [2]:
#Зададим переменные 

target = ['sub_car_claim_click', 'sub_car_claim_submit_click',
'sub_open_dialog_click', 'sub_custom_question_submit_click',
'sub_call_number_click', 'sub_callback_submit_click', 'sub_submit_success',
'sub_car_request_submit_click']

social_network = ['QxAxdyPLuQMEcrdZWdWb', 'MvfHsxITijuriZxsqZqt', 'ISrKoXQCxqqYvAZICvjs',
                  'IZEXUFLARCUMynmHNBGo', 'PlbkrSYoHuZBWfYjYnfw','gVRrcxiDQubJiljoTbGm']

organic = ['organic','referral', '(none)']

In [20]:
#Делим приход пользователей на тех что из соц.сетей и остальные
df_clean["coming"] = pd.Series()
for i in tqdm(range(len(df_clean.utm_source))):
    if df_clean.utm_source[i] in social_network:
        df_clean["coming"][i] = "social"
    else:
        df_clean["coming"][i] = "other" 
              
print(df_clean.coming.value_counts())  

C:\Users\Denis\AppData\Local\Temp\ipykernel_10248\3347580193.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_clean["coming"] = pd.Series()
  0%|          | 0/15684519 [00:00<?, ?it/s]C:\Users\Denis\AppData\Local\Temp\ipykernel_10248\3347580193.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["coming"][i] = "other"
100%|██████████| 15684519/15684519 [02:32<00:00, 102899.08it/s]

other     14481993
social     1202526
Name: coming, dtype: int64


In [21]:
#Делим трафик на две категории - органический и платный
df_clean["traffic"] = pd.Series()
for i in tqdm(range(len(df_clean.utm_medium))):
    if df_clean.utm_medium[i] in organic:
        df_clean["traffic"][i] = "organic"
    else:
        df_clean["traffic"][i] = "paid"
        
print(df_clean.traffic.value_counts())  

C:\Users\Denis\AppData\Local\Temp\ipykernel_10248\451020598.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_clean["traffic"] = pd.Series()
  0%|          | 0/15684519 [00:00<?, ?it/s]C:\Users\Denis\AppData\Local\Temp\ipykernel_10248\451020598.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["traffic"][i] = "paid"
100%|██████████| 15684519/15684519 [02:33<00:00, 102364.85it/s]


paid       9866952
organic    5817567
Name: traffic, dtype: int64


In [22]:
#Делим действие на целевое(1) и нецелевое(0)
df_clean["target"] = pd.Series()
for i in tqdm(range(len(df_clean.event_action))):
    if df_clean.event_action[i] in target:
        df_clean["target"][i] = "1"
    else:
        df_clean["target"][i] = "0"

#Удалим колонку "event_action", она более не нужна.
df_clean = df_clean.drop(["event_action"], axis=1)        

print(df_clean.target.value_counts())  

C:\Users\Denis\AppData\Local\Temp\ipykernel_10248\55128251.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_clean["target"] = pd.Series()
  0%|          | 0/15684519 [00:00<?, ?it/s]C:\Users\Denis\AppData\Local\Temp\ipykernel_10248\55128251.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["target"][i] = "0"
100%|██████████| 15684519/15684519 [02:36<00:00, 100263.92it/s]


0    15581687
1      102832
Name: target, dtype: int64


In [23]:
print(df_clean.shape)
df_clean.head()

(15684519, 17)


,session_id,utm_source,utm_medium,utm_campaign,device_category,device_brand,device_browser,geo_country,geo_city,hit_number,hit_referer,target_city,car_brand,car_model,coming,traffic,target
0,9055434745589932991.1637753792.1637753792,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,mobile,Huawei,chrome,Russia,zlatoust,3,HbolMJUevblAbkHClEQa,spb,none,none,other,paid,0
1,9055434745589932991.1637753792.1637753792,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,mobile,Huawei,chrome,Russia,zlatoust,4,HbolMJUevblAbkHClEQa,spb,none,none,other,paid,0
2,905544597018549464.1636867290.1636867290,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,mobile,Samsung,samsung,Russia,moscow,3,HbolMJUevblAbkHClEQa,moscow,none,none,social,paid,0
3,9055446045651783499.1640648526.1640648526,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,mobile,Huawei,chrome,Russia,krasnoyarsk,10,HbolMJUevblAbkHClEQa,spb,none,none,other,paid,0
4,9055446045651783499.1640648526.1640648526,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,mobile,Huawei,chrome,Russia,krasnoyarsk,13,HbolMJUevblAbkHClEQa,spb,none,none,other,paid,0


### 3.4 Приведение типов данных

In [24]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15684519 entries, 0 to 15684518
Data columns (total 17 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   session_id       object
 1   utm_source       object
 2   utm_medium       object
 3   utm_campaign     object
 4   device_category  object
 5   device_brand     object
 6   device_browser   object
 7   geo_country      object
 8   geo_city         object
 9   hit_number       int64 
 10  hit_referer      object
 11  target_city      object
 12  car_brand        object
 13  car_model        object
 14  coming           object
 15  traffic          object
 16  target           object
dtypes: int64(1), object(16)
memory usage: 2.0+ GB


In [25]:
df_clean.device_category = df_clean.device_category.astype('category')
df_clean.target = df_clean.target.astype('category')
df_clean.traffic = df_clean.traffic.astype('category')
df_clean.coming = df_clean.coming.astype('category')

In [26]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15684519 entries, 0 to 15684518
Data columns (total 17 columns):
 #   Column           Dtype   
---  ------           -----   
 0   session_id       object  
 1   utm_source       object  
 2   utm_medium       object  
 3   utm_campaign     object  
 4   device_category  category
 5   device_brand     object  
 6   device_browser   object  
 7   geo_country      object  
 8   geo_city         object  
 9   hit_number       int64   
 10  hit_referer      object  
 11  target_city      object  
 12  car_brand        object  
 13  car_model        object  
 14  coming           category
 15  traffic          category
 16  target           category
dtypes: category(4), int64(1), object(12)
memory usage: 1.6+ GB


In [27]:
#Сохраняем полученный Дф.
df_clean.to_csv('./data/df_clean.csv', index=False)

# 4. Аналитика

### 4.1 Проверка гипотез

In [3]:
df = pd.read_csv('data/df_clean.csv')
print(df.shape)
df.head()

(15684519, 17)


,session_id,utm_source,utm_medium,utm_campaign,device_category,device_brand,device_browser,geo_country,geo_city,hit_number,hit_referer,target_city,car_brand,car_model,coming,traffic,target
0,9055434745589932991.1637753792.1637753792,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,mobile,Huawei,chrome,Russia,zlatoust,3,HbolMJUevblAbkHClEQa,spb,none,none,other,paid,0
1,9055434745589932991.1637753792.1637753792,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,mobile,Huawei,chrome,Russia,zlatoust,4,HbolMJUevblAbkHClEQa,spb,none,none,other,paid,0
2,905544597018549464.1636867290.1636867290,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,mobile,Samsung,samsung,Russia,moscow,3,HbolMJUevblAbkHClEQa,moscow,none,none,social,paid,0
3,9055446045651783499.1640648526.1640648526,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,mobile,Huawei,chrome,Russia,krasnoyarsk,10,HbolMJUevblAbkHClEQa,spb,none,none,other,paid,0
4,9055446045651783499.1640648526.1640648526,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,mobile,Huawei,chrome,Russia,krasnoyarsk,13,HbolMJUevblAbkHClEQa,spb,none,none,other,paid,0


In [3]:
#Расчет CR для органического трафика
O = df.traffic.value_counts()["organic"]
T = sum((df.traffic == "organic" ) & (df.target == 1 ))
CRo = T/O*100

print(f"Колличество событий органического трафика: {O}")
print(f"Количество событий органического трафика,в которых было выполнено  целевое действие: {T}")
print(f"CR органического трафика = {round(CRo, 2)}%")

Колличество событий органического трафика: 5817567
Количество событий органического трафика,в которых было выполнено  целевое действие: 41672
CR органического трафика = 0.72%


In [4]:
#Расчет CR для платного трафика
P = df.traffic.value_counts()["paid"]
T = sum((df.traffic == "paid" ) & (df.target == 1 ))
CRp = T/P*100

print(f"Колличество событий платного трафика: {P}")
print(f"Количество событий платного трафика,в которых было выполнено  целевое действие: {T}")
print(f"CR платного трафика = {round(CRp, 2)}%")

Колличество событий платного трафика: 9866952
Количество событий платного трафика,в которых было выполнено  целевое действие: 61160
CR платного трафика = 0.62%


#### Органический трафик с точки зрения CR на 0.1% эффективнее платного трафика

In [5]:
#Расчет CR для трафика с мобильных устройств (включая планшеты)
M = df.device_category.value_counts()["mobile"] + df.device_category.value_counts()["tablet"]
Tm = sum((df.device_category == "mobile") & (df.target == 1))
Tt = sum((df.device_category == "tablet") & (df.target == 1))
T = Tm + Tt
CRm = T/M*100

print(f"Колличество событий органического трафика: {M}")
print(f"Количество событий органического трафика,в которых было выполнено  целевое действие: {T}")
print(f"CR органического трафика = {round(CRm, 2)}%")

Колличество событий органического трафика: 11739799
Количество событий органического трафика,в которых было выполнено  целевое действие: 77593
CR органического трафика = 0.66%


In [6]:
#Расчет CR для трафика с десктоптных устройств
D = df.device_category.value_counts()["desktop"]
T = sum((df.device_category == "desktop") & (df.target == 1))
CRd = T/D*100

print(f"Колличество событий органического трафика: {D}")
print(f"Количество событий органического трафика,в которых было выполнено  целевое действие: {T}")
print(f"CR органического трафика = {round(CRd, 2)}%")

Колличество событий органического трафика: 3944720
Количество событий органического трафика,в которых было выполнено  целевое действие: 25239
CR органического трафика = 0.64%


#### Трафик с мобильных устройств ,действительно, практически не отличается от трафика с десктопных устройств с точки зрения CR. Разница составила 0.02% в пользу трафика с мобильных устройств.

In [7]:
#Расчет CR для трафика с Москвы и Санкт-Петербурга
C = df.target_city.value_counts()["moscow"] + df.target_city.value_counts()["spb"]
Tcm = sum((df.target_city == "moscow") & (df.target == 1))
Tcs = sum((df.target_city == "spb") & (df.target == 1))
T = Tcm + Tcs
CRc = T/C*100

print(f"Колличество событий органического трафика: {C}")
print(f"Количество событий органического трафика,в которых было выполнено  целевое действие: {T}")
print(f"CR органического трафика = {round(CRc, 2)}%")

Колличество событий органического трафика: 13319033
Количество событий органического трафика,в которых было выполнено  целевое действие: 89096
CR органического трафика = 0.67%


In [8]:
#Расчет CR для трафика с иных регионов
R = df.target_city.value_counts()["other"]
T = sum((df.target_city == "other") & (df.target == 1))
CRr = T/R*100

print(f"Колличество событий органического трафика: {R}")
print(f"Количество событий органического трафика,в которых было выполнено  целевое действие: {T}")
print(f"CR органического трафика = {round(CRr, 2)}%")

Колличество событий органического трафика: 2365486
Количество событий органического трафика,в которых было выполнено  целевое действие: 13736
CR органического трафика = 0.58%


#### Трафик с Москвы и Санкт-Петербурга с точки зрения CR на 0.09% эффективнее трафика с других регионов.

### 4.1 Ответы на вопросы продуктовой команды

#### Вопрос 1:

In [9]:
#Расчитаем самый целевой трафик с точки зрения объема трафика для различный категорий
print(f"Самый большой объем трафика идёт из источника {df.hit_referer.mode()[0]}, и составляет {round(df.hit_referer.value_counts()[0]/len(df)*100, 1)}% от всего объема трафика")
print(f"Самый большой объем трафика идёт с рекламной компании  {df.utm_campaign.mode()[0]}, и составляет {round(df.utm_campaign.value_counts()[0]/len(df)*100, 1)}% от всего объема трафика")
print(f"Самый большой объем трафика идёт с категорий устройств {df.device_category.mode()[0]}, и составляет {round(df.device_category.value_counts()[0]/len(df)*100, 1)}% от всего объема трафика")
print(f"Самый большой объем трафика идёт с устройств бренда  {df.device_brand.mode()[0]}, и составляет {round(df.device_brand.value_counts()[0]/len(df)*100, 1)}% от всего объема трафика")
print(f"Самый большой объем трафика идёт с города {df.geo_city.mode()[0]}, и составляет {round(df.geo_city.value_counts()[0]/len(df)*100, 1)}% от всего объема трафика")

Самый большой объем трафика идёт из источника HbolMJUevblAbkHClEQa, и составляет 96.4% от всего объема трафика
Самый большой объем трафика идёт с рекламной компании  LTuZkdKfxRGVceoWkVyg, и составляет 47.0% от всего объема трафика
Самый большой объем трафика идёт с категорий устройств mobile, и составляет 73.9% от всего объема трафика
Самый большой объем трафика идёт с устройств бренда  Apple, и составляет 30.0% от всего объема трафика
Самый большой объем трафика идёт с города moscow, и составляет 43.3% от всего объема трафика


#### CR в дальнейшем расчитывается для количества сессий > 1000, потому как при меньшем значении сложно оценить результат.Например, если сессий с города Рязань было всего 2, а целевое действие выполнено 1 раз, CR составит 50%.Но при количестве сессий в 1000, показатель CR  наврятли составит 50%.

In [12]:
#Расчитаем CR для всех источников, количество сессий из которых было больше 1000: 

cr_referer = pd.DataFrame(columns=['Источник', 'Conversion Rate'])

for i in tqdm(range(sum(df.hit_referer.value_counts() > 1000))):
    if df.hit_referer.value_counts()[i] > 1000:
            target = sum((df.hit_referer == df.hit_referer.value_counts().keys()[i]) & (df.target == 1))        
            all = df.hit_referer.value_counts()[i]
            CR = round(target/all*100, 2)
            cr_referer.loc[len(cr_referer.index)]=[df.hit_referer.value_counts().keys()[i], CR]

cr_referer = cr_referer.sort_values(by='Conversion Rate', ascending=False)
cr_referer.reset_index(drop=True, inplace=True)
print(f"Три лучших значений конверсии из источников:\n{cr_referer.at[0,'Источник']} = {cr_referer.at[0,'Conversion Rate']}%, {cr_referer.at[1,'Источник']} = {cr_referer.at[1,'Conversion Rate']}%, {cr_referer.at[2,'Источник']} = {cr_referer.at[2,'Conversion Rate']}%")
cr_referer.head()

100%|██████████| 39/39 [02:18<00:00,  3.54s/it]

Три лучших значений конверсии из источников:
qurtzHmvijzwwngERJfV = 3.3%, IXrZcSwaRZBPpHIybFlM = 2.43%, sAGDBCdeVPmQPwkVEcIX = 2.42%


,Источник,Conversion Rate
0,qurtzHmvijzwwngERJfV,3.30
1,IXrZcSwaRZBPpHIybFlM,2.43
2,sAGDBCdeVPmQPwkVEcIX,2.42
3,FwdMTcXzWAwhtsnMAbhS,2.33
4,ayiFomhbZsohffvJEYjc,2.32


In [13]:
#Расчитаем CR для всех рекламных компаний, количество сессий из которых было больше 1000:

cr_campaign = pd.DataFrame(columns=['Компания', 'Conversion Rate'])

for i in tqdm(range(sum(df.utm_campaign.value_counts() > 1000))):
    if df.utm_campaign.value_counts()[i] > 1000:
            target = sum((df.utm_campaign == df.utm_campaign.value_counts().keys()[i]) & (df.target == 1))        
            all = df.utm_campaign.value_counts()[i]
            CR = round(target/all*100, 2)
            cr_campaign.loc[len(cr_campaign.index)]=[df.utm_campaign.value_counts().keys()[i], CR]

cr_campaign = cr_campaign.sort_values(by='Conversion Rate', ascending=False)
cr_campaign.reset_index(drop=True, inplace=True)
print(f"Три лучших значений конверсии из рекламных компаний:\n{cr_campaign.at[0,'Компания']} = {cr_campaign.at[0,'Conversion Rate']}%, {cr_campaign.at[1,'Компания']} = {cr_campaign.at[1,'Conversion Rate']}%, {cr_campaign.at[2,'Компания']} = {cr_campaign.at[2,'Conversion Rate']}%")
cr_campaign.head()

100%|██████████| 163/163 [09:28<00:00,  3.48s/it]

Три лучших значений конверсии из рекламных компаний:
hJvOVTMdhkqIBqjVrsOL = 8.2%, kwdmElMUPDZaLQdgjcsI = 5.27%, KCcrgoFqYxCpSjdRyJjZ = 4.3%


,Компания,Conversion Rate
0,hJvOVTMdhkqIBqjVrsOL,8.20
1,kwdmElMUPDZaLQdgjcsI,5.27
2,KCcrgoFqYxCpSjdRyJjZ,4.30
3,emlBaVkgLVRqbEVOSSXt,3.58
4,DsqwamlMwRgMTyspCjFR,3.39


In [14]:
#Расчитаем CR для всех категорий устройств:

cr_dc= pd.DataFrame(columns=['Категория устройства', 'Conversion Rate'])

for i in tqdm(range(len(df.device_category.value_counts()))):
    target = sum((df.device_category == df.device_category.value_counts().keys()[i]) & (df.target == 1))        
    all = df.device_category.value_counts()[i]
    CR = round(target/all*100, 2)
    cr_dc.loc[len(cr_dc.index)]=[df.device_category.value_counts().keys()[i], CR]

cr_dc = cr_dc.sort_values(by='Conversion Rate', ascending=False)
cr_dc.reset_index(drop=True, inplace=True)
print(f"Три лучших значений конверсии из категорий устройств:\n{cr_dc.at[0,'Категория устройства']} = {cr_dc.at[0,'Conversion Rate']}%, {cr_dc.at[1,'Категория устройства']} = {cr_dc.at[1,'Conversion Rate']}%, {cr_dc.at[2,'Категория устройства']} = {cr_dc.at[2,'Conversion Rate']}%")
cr_dc.head()

100%|██████████| 3/3 [00:08<00:00,  2.89s/it]

Три лучших значений конверсии из категорий устройств:
mobile = 0.66%, desktop = 0.64%, tablet = 0.53%


,Категория устройства,Conversion Rate
0,mobile,0.66
1,desktop,0.64
2,tablet,0.53


In [15]:
#Расчитаем CR для всех брендов устройств, количество сессий из которых было больше 1000:

cr_db= pd.DataFrame(columns=['Бренд устройства', 'Conversion Rate'])

for i in tqdm(range(sum(df.device_brand.value_counts() > 1000))):
    if df.device_brand.value_counts()[i] > 1000:
        target = sum((df.device_brand == df.device_brand.value_counts().keys()[i]) & (df.target == 1))        
        all = df.device_brand.value_counts()[i]
        CR = round(target/all*100, 2)
        cr_db.loc[len(cr_db.index)]=[df.device_brand.value_counts().keys()[i], CR]

cr_db = cr_db.sort_values(by='Conversion Rate', ascending=False)
cr_db.reset_index(drop=True, inplace=True)
print(f"Три лучших значений конверсии из брендов устройств:\n{cr_db.at[0,'Бренд устройства']} = {cr_db.at[0,'Conversion Rate']}%, {cr_db.at[1,'Бренд устройства']} = {cr_db.at[1,'Conversion Rate']}%, {cr_db.at[2,'Бренд устройства']} = {cr_db.at[2,'Conversion Rate']}%")
cr_db.head()

100%|██████████| 40/40 [02:17<00:00,  3.44s/it]

Три лучших значений конверсии из брендов устройств:
Alcatel = 1.16%, Tecno = 0.95%, LeEco = 0.92%


,Бренд устройства,Conversion Rate
0,Alcatel,1.16
1,Tecno,0.95
2,LeEco,0.92
3,itel,0.91
4,OPPO,0.88


In [16]:
#Расчитаем CR для всех городов, количество сессий из которых было больше 1000:

cr_gc= pd.DataFrame(columns=['Город', 'Conversion Rate'])

for i in tqdm(range(sum(df.geo_city.value_counts() > 1000))):
    if df.geo_city.value_counts()[i] > 1000:
        target = sum((df.geo_city == df.geo_city.value_counts().keys()[i]) & (df.target == 1))        
        all = df.geo_city.value_counts()[i]
        CR = round(target/all*100, 2)
        cr_gc.loc[len(cr_gc.index)]=[df.geo_city.value_counts().keys()[i], CR]

cr_gc = cr_gc.sort_values(by='Conversion Rate', ascending=False)
cr_gc.reset_index(drop=True, inplace=True)
print(f"Три лучших значений конверсии из городов:\n{cr_gc.at[0,'Город']} = {cr_gc.at[0,'Conversion Rate']}%, {cr_gc.at[1,'Город']} = {cr_gc.at[1,'Conversion Rate']}%, {cr_gc.at[2,'Город']} = {cr_gc.at[2,'Conversion Rate']}%")
cr_gc.head()

100%|██████████| 318/318 [18:54<00:00,  3.57s/it]

Три лучших значений конверсии из городов:
domodedovo = 1.97%, tallinn = 1.36%, nazran = 1.24%


,Город,Conversion Rate
0,domodedovo,1.97
1,tallinn,1.36
2,nazran,1.24
3,side,1.22
4,dimitrovgrad,1.22


#### Вопрос 2:

In [17]:
#Расчитаем CR для автомобилей, количество сессий для которых было больше 1000:

cr_cb= pd.DataFrame(columns=['Автомобиль', 'Conversion Rate'])

for i in tqdm(range(sum(df.car_brand.value_counts() > 1000))):
    if df.car_brand.value_counts()[i] > 1000:
        target = sum((df.car_brand == df.car_brand.value_counts().keys()[i]) & (df.target == 1))        
        all = df.car_brand.value_counts()[i]
        CR = round(target/all*100, 2)
        cr_cb.loc[len(cr_cb.index)]=[df.car_brand.value_counts().keys()[i], CR]

cr_cb = cr_cb.sort_values(by='Conversion Rate', ascending=False)
cr_cb.reset_index(drop=True, inplace=True)
print(f"Три лучших значений конверсии для автомобилей:\n{cr_cb.at[0,'Автомобиль']} = {cr_cb.at[0,'Conversion Rate']}%, {cr_cb.at[1,'Автомобиль']} = {cr_cb.at[1,'Conversion Rate']}%, {cr_cb.at[2,'Автомобиль']} = {cr_cb.at[2,'Conversion Rate']}%")
cr_cb.head()

100%|██████████| 21/21 [01:16<00:00,  3.64s/it]

Три лучших значений конверсии для автомобилей:
hyundai = 1.89%, lada-vaz = 1.3%, volkswagen = 1.19%


,Автомобиль,Conversion Rate
0,hyundai,1.89
1,lada-vaz,1.30
2,volkswagen,1.19
3,kia,1.15
4,haval,1.11


#### Лучшие показатели конверсии имеют такие марки автомобилей как: Hyundai, Lada и Volkswagen. Пользователи сервиса чаще обращают внимание на бюджетные автомобили. Возможно, следует планировать пополнение автопарка с учетом данных показателей.

#### Вопрос 3:

In [27]:
#Расчитаем CR для прихода с соц.сетей:
S = df.utm_source.value_counts()[social_network]
cr_coming= pd.DataFrame(columns=['Соц.сеть', 'Conversion Rate'])

for i in tqdm(range(len(S))):
    target = sum((df.utm_source == S.keys()[i]) & (df.target == 1))        
    all = S[i]
    CR = round(target/all*100, 2)
    cr_coming.loc[len(cr_coming.index)]=[S.keys()[i], CR]

cr_coming = cr_coming.sort_values(by='Conversion Rate', ascending=False)
cr_coming.reset_index(drop=True, inplace=True)
print(f"Три лучших значений конверсии для соц.сетей:\n{cr_coming.at[0,'Соц.сеть']} = {cr_coming.at[0,'Conversion Rate']}%, {cr_coming.at[1,'Соц.сеть']} = {cr_coming.at[1,'Conversion Rate']}%, {cr_coming.at[2,'Соц.сеть']} = {cr_coming.at[2,'Conversion Rate']}%")
cr_coming.head(6)

100%|██████████| 6/6 [00:07<00:00,  1.22s/it]

Три лучших значений конверсии для соц.сетей:
MvfHsxITijuriZxsqZqt = 0.75%, IZEXUFLARCUMynmHNBGo = 0.75%, QxAxdyPLuQMEcrdZWdWb = 0.68%


,Соц.сеть,Conversion Rate
0,MvfHsxITijuriZxsqZqt,0.75
1,IZEXUFLARCUMynmHNBGo,0.75
2,QxAxdyPLuQMEcrdZWdWb,0.68
3,gVRrcxiDQubJiljoTbGm,0.49
4,ISrKoXQCxqqYvAZICvjs,0.37
5,PlbkrSYoHuZBWfYjYnfw,0.17


In [32]:
#Получим список всех источников, кроме соц. сетей.
other_sources = list(set(df.utm_source.tolist()))
for i in range(len(social_network)):
    other_sources.remove(social_network[i])

print(other_sources)

['FTAuYVNoYYxgvKMpKSLW',
 'ySpKPVDYCgmkHVSeLjnJ',
 'CXgqTLNTvvxWUWoOfjNF',
 'MlvrtLODeciGJoIzwoNf',
 'gSwaXkJgHiWoIntUiupK',
 'RAeIjhqmnmNXpLaHKdgp',
 'aXQzDWsJuGXeBXexNHjc',
 'sOTrgtUMUuYzzWBPLeFQ',
 'vEBWkLmuIpqNdxZZdlnW',
 'LBaWJEeycBIqEqUqkoqw',
 'hYVrCkhCPSqKBhZYhWVq',
 'EyveFmCGCQchHbSIRpVc',
 'xQnKzPSrehWizZOmwHkM',
 'CHdbSXINvRmKaAtGIGzz',
 'yCnTwlCLkqZHwAglrEll',
 'FTjNLDyTrXaWYgZymFkV',
 'tYfqTBjAxWbmBruissGm',
 'ngkgBNjlzLYBofkljaBo',
 'azajeHUvPOKkHBvWFbjz',
 'eooqChmKKdvCvkvVEKSN',
 'ysUnXJVzkMTkkmGifVJI',
 'MEitHbTnBcZIHbJvtCyh',
 'RPdScaVSDDKvfbqPGRVe',
 'RzLAoRYmCtVATSoPvWAQ',
 'TTtiRKFZIaQpIWggfCoF',
 'GWZGdHKPgmZPNPwkJshU',
 'LkGnzVRewoaOHnMCwadT',
 'fUhxIMXvMivfzvXbtxXS',
 'RmEBuqrriAfAVsLQQmhk',
 'GkGprIZHXAKqgsyDIvMR',
 'fJCYsujgSxIHFbOmgDdN',
 'VCREhgqUPSUkmfOTvGiW',
 'QuIQZzbAETTwBxLvIiyY',
 'eimRuUrNhZLAYcwRrNXu',
 'ShCnUddaAbEvEVCCZyOt',
 'DsqwamlMwRgMTyspCjFR',
 'MYrKypXYDhDVwJMUhCqu',
 'JcVHXwVSrnfIuOMMEkrJ',
 'XGOsamgBesNClOIokPJY',
 'UphNUPPFIJLIZnvubKDj',


In [49]:
#Расчитаем CR для прихода из других источников:
L = df.utm_source.value_counts()[other_sources]
cr_other= pd.DataFrame(columns=['Источник', 'Conversion Rate'])

for i in tqdm(range(sum(L > 1000))):
    if df.utm_source.value_counts()[i] > 1000:
        target = sum((df.utm_source == L.keys()[i]) & (df.target == 1))        
        all = L[i]
        CR = round(target/all*100, 2)
        cr_other.loc[len(cr_other.index)]=[L.keys()[i], CR]

cr_other = cr_other.sort_values(by='Conversion Rate', ascending=False)
cr_other.reset_index(drop=True, inplace=True)
print(f"Три лучших значений конверсии для соц.сетей:\n{cr_other.at[0,'Источник']} = {cr_other.at[0,'Conversion Rate']}%, {cr_other.at[1,'Источник']} = {cr_other.at[1,'Conversion Rate']}%, {cr_other.at[2,'Источник']} = {cr_other.at[2,'Conversion Rate']}%")
cr_other.head()

100%|██████████| 59/59 [01:47<00:00,  1.82s/it]

Три лучших значений конверсии для соц.сетей:
XzfzEBYZWgSDtJNXOadn = 25.0%, fJCYsujgSxIHFbOmgDdN = 22.94%, JcVHXwVSrnfIuOMMEkrJ = 11.11%


,Источник,Conversion Rate
0,XzfzEBYZWgSDtJNXOadn,25.00
1,fJCYsujgSxIHFbOmgDdN,22.94
2,JcVHXwVSrnfIuOMMEkrJ,11.11
3,hONgZhBaHwLGhvJSefNE,5.56
4,MlvrtLODeciGJoIzwoNf,3.12


#### CR от прихода с соц. сетей составляет от 0,17 до 0,75%, в то время как CR от прихода из других источников доходит до 25%. По моему мнению, не стоит увеличивать рекламную компанию в соц. сетях. Лучше сосредоточить ресурсы на другие источники. 